# Gender Discrimination and Promotions in a Labor Search Model

## Replication of Flabbi 2010 with an extension to the model

### Presented by Megan McCoy

This is a sub- slide

## Outline

- Flabbi 2010 Model Overview

- Estimation with CPS (1995)

- Estimation with SOEP (2017)

- Extension of the Model 

## Flabbi 2010 Overview

- Continuous time search-match-bargaining model
    - Common discount rate $\rho$
    - Meet at arrival rate $\lambda$, termination at rate $\eta$
    - Match-specific productivity $x \sim G(x)$ realized when meeting
    - Wages by Nash bargaining with power $\alpha$
    - Flow (dis)utility of unemployment $b$

- Workers $\in$ { Male, Female }

- Firms $\in$ { Prejudiced, Not Prejudiced }

    - Prejudiced employers are of proportion $p$
    - Experience a disutility flow $d$ of working with women.

## Flabbi 2010 Model Overview

- The model shows that wage discrimination occurs for multiple reasons:

    - Matching with a prejudiced employer increases the reservation value of the match-specific productivity value and shifts down the wage schedule for women
    - The presence of prejudiced employers lowers value of unemployment for women compared to men 
    
    
- Data from Current Population Survey (CPS) in 1995
    - Age 30-55
    - Employed or looking for employment 
    - White men and women     
    - College educated or more 

## Flabbi 2010 Model: Identification

Wage Equations

$$w_{JI} = \alpha (x - d \textbf{I}_{F,P}) + (1-\alpha) \rho U_J$$

Reservation Wage Equations

$$\rho U_J = b_J + \frac{\lambda_J\alpha}{\rho + \eta_J}\left( p\int [x - d \textbf{I}_{F,P} - \rho U_J]g_J(x)dx = (1-p) \int[x-\rho U_J]g_J(x)dx \right)$$

Hazard Rate Out of Unemployment

$$h_J = \lambda_J\left((1-p)\tilde{G}_J(\rho U_J) + p\tilde{G}_J(\rho U_J + d\textbf{I}_{F})\right)$$

## Flabbi 2010 Model: Estimation

- Reservation Wage 

$$\hat{\rho U_J} = \min_{w_i}\{w_i, i\in E_J\}$$

- Hazard Rate

$$\hat{h_J} = \frac{N_{UJ}}{\sum_{i \in U_J} t_i}$$

- Termination Rate

$$ \hat{\eta_J} = \frac{N_{UJ}}{N_{EJ}}\hat{h_J}$$

```python
## Parameters without distributional assumptions

wstarM = min(ME['wage'])
wstarF = min(FE['wage'])

hM = MU['dur'].count()/sum(MU.values[:,0])
hF = FU['dur'].count()/sum(FU.values[:,0])
h = U['dur'].count()/sum(U.values[:,0])

ηM = hM * (MU['dur'].count()/ME['empl'].count())
ηF = hF * (FU['dur'].count()/FE['empl'].count())
η = h * (U['dur'].count()/E['empl'].count())

α = 0.5
```

```python
# Equations for λ

def lambdaM(h: float, wstarM: float, α: float, μ: float, σ: float):
    """
    Estimates lambda for men
    """
    
    l = (α*μ) + ((1-α)*wstarM)
    s = α * σ
    shape = 1
    
    sf_in = (wstarM-l)/s
    
    denom = stats.lognorm.sf(sf_in, shape, l, s)
    
    return h/denom

def lambdaF(h: float, wstarF: float, α: float, μ: float, σ: float, p: float):
    """
    Estimates lambda for women
    """
    
    l1 = (α*μ) + ((1-α)*wstarF)
    l2 = (α*μ) + ((1-α)*wstarF) - α*d
    s = α * σ
    shape = 1
    
    sf_in1 = (wstarF-l1)/s
    sf_in2 = (wstarF-l2)/s
    
    denom = (1-p)*stats.lognorm.sf(sf_in1, shape, l1, s) + p*stats.lognorm.sf(sf_in2, shape, l2, s)
    
    return h/denom
```

## Flabbi 2010 Model: Identification

Unconditional Unemployment Contribution 

$$f_u(t_i|i \in U|J) = f_u(t_i|i \in U,J)P(i\in U|J) = h_J \exp(-h_Jt_i)\frac{\eta_J}{\eta_J+h_J}$$

Employed Contribution 

$$f_e(w_i, w_i>\rho U_J, i\in E |J)$$

$$= f_e(w_i|w_i>\rho U_J, i\in E, J)P(w_i>\rho U_J|i\in E,J) P(i\in E|J)$$

$$= \left(\frac{\frac{(1-p)}{\alpha}g_J \left( \frac{w_i-(1-\alpha)\rho U_J}{\alpha} \right)}{\tilde{G}_J(\rho U_J)} + \frac{\frac{p}{\alpha}g_J \left( \frac{w_i + \alpha d \textbf{I}_{F,P} -(1-\alpha)\rho U_J}{\alpha} \right)}{\tilde{G}_J(\rho U_J + d \textbf{I}_{F,P})} \right)\frac{h_J}{h_J+\eta_J}$$

## Flabbi 2010 Model: Estimation 

- Distributional Assumption: Lognormal Distribution 

    $$g_J(x,\mu_J,\sigma_J) = \frac{1}{\sigma_J} f \left( \frac{x-\mu_J}{\sigma_J} \right)$$

    - Density of accepted wages for men 
    
    $$ \frac{\frac{1}{s_M}f\left(\frac{w_i-l_M}{s_M} \right)}{\tilde{F}\left( \frac{\rho U_M - l_M}{s_M} \right)} $$
    
    $$l_M \equiv \alpha\mu_M + (1-\alpha)\rho U_M \hspace{2cm} s_M \equiv \alpha \sigma_M$$
    
    - Density of accepted wages for women 
    
    $$ (1-p) \frac{\frac{1}{s_{FN}}f\left(\frac{w_i-l_{FN}}{s_{FN}} \right)}{\tilde{F}\left( \frac{\rho U_F - l_{FN}}{s_{FN}} \right)} + p \frac{\frac{1}{s_{FP}}f\left(\frac{w_i-l_{FP}}{s_{FP}} \right)}{\tilde{F}\left( \frac{\rho U_{FP} - l_{FP}}{s_{}FP} \right)} $$  
    
    $$l_{FN} \equiv \alpha\mu_F + (1-\alpha)\rho U_F \hspace{1cm} l_{FP} \equiv \alpha\mu_F + (1-\alpha)\rho U_F - \alpha d \hspace{1cm} s_{FN} = s_{FP} \equiv \alpha \sigma_F$$


```python
# Accepted Wage Function and Distributions

def dens_accepted(wage: np.array, α: float, μ: float, σ: float, wstar: float):
    """
    Calculates the density of accepted wages using the lognormal distribution
    """
    
    l = (α*μ) + ((1-α)*wstar)
    s = α * σ
    shape = 1
    
    sf_in = (wstar - l)/s
    
    return stats.lognorm.pdf(wage, shape, l, s) / stats.lognorm.sf(sf_in, shape, l, s)

def dens_accepted_prej(wage: np.array, α: float, μ: float, σ: float, wstar: float, d: float):
    """
    Calculates the density of accepted wages when prejudice is present using the lognormal distribution
    """
    
    l = (α*μ) + ((1-α)*wstar) - α*d
    s = α * σ
    shape = 1
    
    sf_in = (wstar - l)/s
    
    return stats.lognorm.pdf(wage,shape,l,s) / stats.lognorm.sf(sf_in, shape, l, s)
```

## Flabbi 2010 Model: Identification

<img src='./figures/FlabbiLogL.png'>

```python
def loglik_6( params: list ):
    """
    Calculates log likelihood with prejudice and productivity differences 
    """
    λM = np.exp(params[0])
    λF = np.exp(params[1])
    ηM = np.exp(params[2])
    ηF = np.exp(params[3])
    μM = params[4]
    σM = np.exp(params[5])
    μF = params[6]
    σF = np.exp(params[7])
    d = np.exp(params[8])
    p = np.exp(params[9])/(1+np.exp(params[9]))

    # Men's equations 
    λM = lambdaM(hM, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(hM/(hM+ηM))
    b = MU['dur'].count() * np.log(ηM)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μM, σM, wstarM) ) )
    
    # Women's equations
    λF = lambdaF(hF, wstarF, α, μM, σM, p)
    
    f = F['dur'].count() * np.log(hF/(hF+ηF))
    g = FU['dur'].count() * np.log(ηF)
    h = - hF * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted(FE['wage'], α, μF, σF, wstarF)
    z = (p/α) * dens_accepted_prej(FE['wage'], α, μF, σF, wstarF, d)

    i = np.sum( np.log( y + z ) )
    
    return (a + b + c + e + f + g + h + i)
```

```python
# Check on log-likelihood

param6 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, μ_F, σ_F, d, p]

b6_0 = np.log(param6[0])
b6_1 = np.log(param6[1])
b6_2 = np.log(param6[2])
b6_3 = np.log(param6[3])
b6_4 = param6[4]
b6_5 = np.log(param6[5])
b6_6 = param6[6]
b6_7 = np.log(param6[7])
b6_8 = np.log(param6[8])
b6_9 = np.log(1)

init6 = [b6_0, b6_1, b6_2, b6_3, b6_4, b6_5, b6_6, b6_7, b6_8, b6_9]
```

```python
# Coefficients

λM_6 = np.exp(est_6.x[0])
λF_6 = np.exp(est_6.x[1])
ηM_6 = np.exp(est_6.x[2])
ηF_6 = np.exp(est_6.x[3])
μM_6 = est_6.x[4]
σM_6 = np.exp(est_6.x[5])
μF_6 = est_6.x[6]
σF_6 = np.exp(est_6.x[7])
d_6 = np.exp(est_6.x[8])
p_6 = np.exp(est_6.x[9])/(1+np.exp(est_6.x[9]))

coeff_6 = [λM_6, λF_6, ηM_6, ηF_6, μM_6, σM_6, μF_6, σF_6, d_6, p_6]

```

```python
# Function for calculating standard errors and significance

def teststats (hess_inv : np.ndarray, lnL : float, nparams : int):
    """
    Calculates the standard errors and p value from the LR tests
    """
    se = np.sqrt(np.diag(hess_inv))
    
    loglik_H0 = lnL_6
    
    LR = 2 * (lnL - loglik_H0)
    pval = stats.chi2.pdf(LR, nparams)
    
    return [se.tolist(), [pval]]
```

## Estimation with CPS (1995)

<img src='./figures/CPSdescr.png'>

## Output from Flabbi 2010

<img src='./figures/Flabbi_Descr.png'>

## Estimation with CPS (1995)

<img src='./figures/fig1_2.png'>

## Figure from Flabbi 2010, Men's Distribution

<img src='./figures/FlabbiMenDistr.png'>

## Figure from Flabbi 2010, Women Distribution 

<img src='./figures/FlabbiWomenDistr.png'>

## Estimation Results with CPS (1995)

<img src='./figures/CPSest.png' width="600">

## Output from Flabbi 2010 

<img src='./figures/FlabbiMLE.png'  width="600">

## Estimation with SOEP (2017)

- Socio-Economic Panel (SOEP) is collected by the German Institute for Economic Research 

- Data from 2017
    - Age 30-55
    - Employed full-time or unemployed

## Estimation with SOEP (2017)

<img src='./figures/SOEPdescr.png'>

## Estimation with SOEP (2017)

<img src='./figures/fig1_2_SOEP.png'>

## Extension of the Model

- Stationary, continuous time search-match-bargaining model
    - Common discount rate $\rho$ and death rate $\delta$
    - Meet at arrival rate $\lambda$, termination at rate $\eta$
    - Match-specific productivity $x \sim G(x)$ realized when meeting
    - Wages by Nash bargaining with power $\alpha$
    - Flow (dis)utility of unemployment $b$
   
- Workers $\in$ { Male, Female }
- Firms $\in$ {Prejudiced, Not Prejudiced }
    - Prejudiced employers are of proportion $p$
    - Experience a disutility flow $d$ of working with women.

- Include promotion shocks
    - Employees start in the labor market at position level $a_0$
    - When employed, they receive a promotion shock $\tau_{\textbf{I}_{\{F,P\}},k}$ to level $k'>k$
    - The position shock does not depreciate
    - Productivity $y(k,x) = a_k x$

## Value Functions

## Wages

## Equilibrium

## Identification Strategy

## Going Forward